In [1]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv', encoding = 'cp949')
df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202307,1호선,동대문,710,31,11581,2095,9027,6829,14177,...,8930,511,2412,0,216,0,0,0,0,20230803
1,202307,1호선,동묘앞,221,2,3012,1090,3682,5042,7336,...,4195,104,1649,0,272,0,0,0,0,20230803
2,202307,1호선,서울역,577,37,8434,9396,13088,51243,40143,...,13987,2297,2598,1,219,0,0,0,0,20230803
3,202307,1호선,시청,49,5,2381,4825,3784,22736,7558,...,4285,287,1085,0,12,0,0,0,0,20230803
4,202307,1호선,신설동,392,35,8967,2356,9520,8509,21822,...,9505,317,2244,0,0,0,0,0,0,20230803


- 서울시 지하철 승하차인원의 월별 합계
- 2018년 이후 2호선 승차 인원에 대해서만 분석 실시

## 질문
- 승차 인원이 가장 많은 역은?
- 연도별로 혹은 월별로 승차 인원 추이에 차이가 있는가?
- 시간대별로 가장 승차인원이 많은 역은?
- 지하철역 시간대별 인원 유형 군집화
- 지도에 분석 결과를 시각화하기

## 전처리

#### 1) 연도, 월 컬럼 추가

In [2]:
df['연도'] = pd.to_datetime(df['사용월'], format = '%Y%m').dt.year
df['월'] = pd.to_datetime(df['사용월'], format = '%Y%m').dt.month

df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자,연도,월
0,202307,1호선,동대문,710,31,11581,2095,9027,6829,14177,...,2412,0,216,0,0,0,0,20230803,2023,7
1,202307,1호선,동묘앞,221,2,3012,1090,3682,5042,7336,...,1649,0,272,0,0,0,0,20230803,2023,7
2,202307,1호선,서울역,577,37,8434,9396,13088,51243,40143,...,2598,1,219,0,0,0,0,20230803,2023,7
3,202307,1호선,시청,49,5,2381,4825,3784,22736,7558,...,1085,0,12,0,0,0,0,20230803,2023,7
4,202307,1호선,신설동,392,35,8967,2356,9520,8509,21822,...,2244,0,0,0,0,0,0,20230803,2023,7


In [3]:
df = df.query('연도 >= 2018 and 호선명 == "2호선"')

df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자,연도,월
10,202307,2호선,강남,120,14,9359,10930,18503,56268,40112,...,6609,3,47,0,0,0,0,20230803,2023,7
11,202307,2호선,강변(동서울터미널),27,1,8936,2251,28877,21987,78164,...,5412,1,39,0,0,0,0,20230803,2023,7
12,202307,2호선,건대입구,290,1,16965,1813,22074,15573,53127,...,10526,3,862,0,0,0,0,20230803,2023,7
13,202307,2호선,교대(법원.검찰청),15,0,2693,6957,11535,25054,25250,...,3275,2,15,0,0,0,0,20230803,2023,7
14,202307,2호선,구로디지털단지,369,39,38505,5472,53882,20639,144838,...,14233,6,308,0,0,0,0,20230803,2023,7


#### 2) 지하철역 이름 통일

In [4]:
df['지하철역'].unique()

array(['강남', '강변(동서울터미널)', '건대입구', '교대(법원.검찰청)', '구로디지털단지', '구의(광진구청)',
       '낙성대(강감찬)', '당산', '대림(구로구청)', '도림천', '동대문역사문화공원(DDP)', '뚝섬', '문래',
       '방배', '봉천', '사당', '삼성(무역센터)', '상왕십리', '서울대입구(관악구청)', '서초', '선릉',
       '성수', '시청', '신답', '신당', '신대방', '신도림', '신림', '신설동', '신정네거리', '신촌',
       '아현', '양천구청', '역삼', '영등포구청', '왕십리(성동구청)', '용답', '용두(동대문구청)',
       '을지로3가', '을지로4가', '을지로입구', '이대', '잠실(송파구청)', '잠실나루', '잠실새내',
       '종합운동장', '충정로(경기대입구)', '한양대', '합정', '홍대입구', '낙성대', '동대문역사문화공원'],
      dtype=object)

In [5]:
df['지하철역'] = [i[0] for i in df['지하철역'].str.split('(')]
df['지하철역'].unique()

array(['강남', '강변', '건대입구', '교대', '구로디지털단지', '구의', '낙성대', '당산', '대림',
       '도림천', '동대문역사문화공원', '뚝섬', '문래', '방배', '봉천', '사당', '삼성', '상왕십리',
       '서울대입구', '서초', '선릉', '성수', '시청', '신답', '신당', '신대방', '신도림', '신림',
       '신설동', '신정네거리', '신촌', '아현', '양천구청', '역삼', '영등포구청', '왕십리', '용답',
       '용두', '을지로3가', '을지로4가', '을지로입구', '이대', '잠실', '잠실나루', '잠실새내',
       '종합운동장', '충정로', '한양대', '합정', '홍대입구'], dtype=object)

#### 3) 승하차인원에서 승차 인원만 추출

In [6]:
df.columns

Index(['사용월', '호선명', '지하철역', '04시-05시 승차인원', '04시-05시 하차인원', '05시-06시 승차인원',
       '05시-06시 하차인원', '06시-07시 승차인원', '06시-07시 하차인원', '07시-08시 승차인원',
       '07시-08시 하차인원', '08시-09시 승차인원', '08시-09시 하차인원', '09시-10시 승차인원',
       '09시-10시 하차인원', '10시-11시 승차인원', '10시-11시 하차인원', '11시-12시 승차인원',
       '11시-12시 하차인원', '12시-13시 승차인원', '12시-13시 하차인원', '13시-14시 승차인원',
       '13시-14시 하차인원', '14시-15시 승차인원', '14시-15시 하차인원', '15시-16시 승차인원',
       '15시-16시 하차인원', '16시-17시 승차인원', '16시-17시 하차인원', '17시-18시 승차인원',
       '17시-18시 하차인원', '18시-19시 승차인원', '18시-19시 하차인원', '19시-20시 승차인원',
       '19시-20시 하차인원', '20시-21시 승차인원', '20시-21시 하차인원', '21시-22시 승차인원',
       '21시-22시 하차인원', '22시-23시 승차인원', '22시-23시 하차인원', '23시-24시 승차인원',
       '23시-24시 하차인원', '00시-01시 승차인원', '00시-01시 하차인원', '01시-02시 승차인원',
       '01시-02시 하차인원', '02시-03시 승차인원', '02시-03시 하차인원', '03시-04시 승차인원',
       '03시-04시 하차인원', '작업일자', '연도', '월'],
      dtype='object')

In [7]:
on_col = [i for i in df.columns if '승차' in i]
df = df[['사용월', '연도', '월', '지하철역'] + on_col]

df.head()

,사용월,연도,월,지하철역,04시-05시 승차인원,05시-06시 승차인원,06시-07시 승차인원,07시-08시 승차인원,08시-09시 승차인원,09시-10시 승차인원,...,18시-19시 승차인원,19시-20시 승차인원,20시-21시 승차인원,21시-22시 승차인원,22시-23시 승차인원,23시-24시 승차인원,00시-01시 승차인원,01시-02시 승차인원,02시-03시 승차인원,03시-04시 승차인원
10,202307,2023,7,강남,120,9359,18503,40112,64256,52713,...,327480,215011,172551,196756,177694,80992,13773,3,0,0
11,202307,2023,7,강변,27,8936,28877,78164,117728,82763,...,61291,43557,32792,27539,19267,7874,1058,1,0,0
12,202307,2023,7,건대입구,290,16965,22074,53127,91248,61438,...,85337,64419,65838,72636,74202,45555,7559,3,0,0
13,202307,2023,7,교대,15,2693,11535,25250,36820,27461,...,133358,62654,51239,53658,42819,15754,3171,2,0,0
14,202307,2023,7,구로디지털단지,369,38505,53882,144838,184498,112257,...,210532,101581,65184,55202,41572,18845,3463,6,0,0


In [8]:
# df.to_csv('../data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보_final.csv', index = False, encoding = 'UTF-8-sig')